# End-to-End Benchmark

Run QuASAr and baseline backends on benchmark circuits and compare performance.

In [ ]:
import pandas as pd
from benchmarks.runner import BenchmarkRunner
from benchmarks.backends import StatevectorAdapter, DecisionDiagramAdapter, MPSAdapter, StimAdapter
from benchmarks import circuits
from quasar.simulation_engine import SimulationEngine
from quasar_convert import ConversionEngine
import time, matplotlib.pyplot as plt

In [ ]:
class TrackingConversionEngine(ConversionEngine):
    def __init__(self):
        super().__init__()
        self.total_time = 0.0
    def _timeit(self, func, *args, **kwargs):
        start = time.perf_counter()
        res = func(*args, **kwargs)
        self.total_time += time.perf_counter() - start
        return res
    def convert_boundary_to_statevector(self, ssd):
        return self._timeit(super().convert_boundary_to_statevector, ssd)
    def convert_boundary_to_tableau(self, ssd):
        if hasattr(super(), "convert_boundary_to_tableau"):
            return self._timeit(super().convert_boundary_to_tableau, ssd)
        raise AttributeError
    def convert_boundary_to_dd(self, ssd):
        if hasattr(super(), "convert_boundary_to_dd"):
            return self._timeit(super().convert_boundary_to_dd, ssd)
        raise AttributeError
    def extract_local_window(self, state, qubits):
        return self._timeit(super().extract_local_window, state, qubits)
    def build_bridge_tensor(self, left, right):
        return self._timeit(super().build_bridge_tensor, left, right)

In [ ]:
circuit_fns = {
    'ghz': circuits.ghz_circuit,
    'qft': circuits.qft_circuit,
    'w_state': circuits.w_state_circuit,
    'grover': circuits.grover_circuit,
}
backends = {
    'statevector': StatevectorAdapter(),
    'mqt_dd': DecisionDiagramAdapter(),
    'mps': MPSAdapter(),
    'stim': StimAdapter(),
}
REPETITIONS = 5
NUM_QUBITS = 2
records = []
for cname, cfn in circuit_fns.items():
    circuit = cfn(NUM_QUBITS)
    for bname, backend in backends.items():
        for _ in range(REPETITIONS):
            runner = BenchmarkRunner()
            try:
                rec = runner.run(circuit, backend, return_state=False)
            except NotImplementedError:
                continue
            rec.update({'circuit': cname, 'backend_switches': 0, 'conversion_time': 0.0})
            records.append(rec)
    for _ in range(REPETITIONS):
        ce = TrackingConversionEngine()
        runner = BenchmarkRunner()
        engine = SimulationEngine(conversion_engine=ce)
        rec = runner.run_quasar(circuit, engine)
        rec.update({'circuit': cname,
                    'backend_switches': len(rec['result'].conversions),
                    'conversion_time': ce.total_time})
        records.append(rec)
df = pd.DataFrame(records)
df['runtime'] = df['total_time']
df['peak_memory'] = df[['prepare_peak_memory','run_peak_memory']].max(axis=1)
summary = df.groupby(['circuit','framework']).agg(
    runtime_mean=('runtime','mean'), runtime_std=('runtime','std'),
    peak_memory_mean=('peak_memory','mean'), peak_memory_std=('peak_memory','std'),
    backend_switches_mean=('backend_switches','mean'),
    backend_switches_std=('backend_switches','std'),
    conversion_time_mean=('conversion_time','mean'),
    conversion_time_std=('conversion_time','std')
).reset_index()
summary

In [ ]:
speedup = []
for cname in circuit_fns:
    quasar_time = summary[(summary.circuit==cname)&(summary.framework=='quasar')]['runtime_mean'].iloc[0]
    baseline_times = summary[(summary.circuit==cname)&(summary.framework!='quasar')]['runtime_mean']
    if not baseline_times.empty:
        best_baseline = baseline_times.min()
        speedup.append({'circuit': cname, 'speedup': best_baseline/quasar_time})
speedup_df = pd.DataFrame(speedup)
speedup_df

In [ ]:
ax = speedup_df.plot.bar(x='circuit', y='speedup', legend=False)
ax.set_ylabel('QuASAr speedup over best baseline')
ax.set_xlabel('Circuit')
plt.tight_layout()